In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sys
%matplotlib inline
from src import utils

## Questions
<ol>
 <li>Which neighbourhoods are most frequented?</li>
 <li>which elements influence the price?</li>
 <li> What about economy due to COVID-19 quarantine? 
</li>
</ol>

Future works:
<ol>
<li> Satisfation in a review is about perfection or surprise? Sentimental analysis in reviews text
<li> Analysis in prob to rent places (anomaly detection?)
</ol>

# Load Datasets

In [ ]:
# Calendar of next books since 25/10/2020
calendar = pd.read_csv('data/raw/calendar.csv')

# List of places registered in Airbnb Rio
listings = pd.read_csv('data/raw/listings.csv')

# Reviews registered
reviews = pd.read_csv('data/raw/reviews.csv')

# List of neighbourhoods
nb = pd.read_csv('data/raw/neighbourhoods.csv')

# Summary of places registered in Airbnb RIO
lists = pd.read_csv('data/raw/listings_summary.csv')

# Summary of reviews registered
revs = pd.read_csv('data/raw/reviews_summary.csv')

# Info geospatial about Rio
df_places = gpd.read_file('data/raw/neighbourhoods.geojson')

# Filter from listings inative hosts

In [ ]:
listings_filtered = utils.calc_delta_day(listings, 'last_review')

- Filter only hosts who has at least 1 review last 6 months

In [ ]:
listings_filtered = listings_filtered[listings_filtered.delta_last_review <= 180]

# 1. Which neighbourhoods are most frequented?

In [ ]:
count_revs = revs[revs.date >= '2020-04-25'].groupby('listing_id').count().reset_index()

In [ ]:
count_revs.columns = ['id', 'count_reviews']

In [ ]:
neighbourhood = lists[['id', 'neighbourhood']].copy()

In [ ]:
df_neighbourhood = neighbourhood.merge(count_revs, how='inner', on='id')

In [ ]:
count_neighbourhood = df_neighbourhood.groupby('neighbourhood')['count_reviews'].sum().sort_values(ascending=False)

In [ ]:
total_reviews = df_neighbourhood['count_reviews'].sum()

In [ ]:
result_1 = (count_neighbourhood/total_reviews).head(10)
result_1

In [ ]:
result_1.reset_index().neighbourhood

In [ ]:
result_1.reset_index().count_reviews

In [ ]:
plt.rcParams["figure.figsize"] = (12,5)

In [ ]:
plt.bar(result_1.reset_index().neighbourhood.values, result_1.reset_index().count_reviews.values)
plt.xticks(rotation=30)

- Places in the list

In [ ]:
df_neighbourhood.neighbourhood.value_counts(normalize=True).head(10)

- plots

In [ ]:
# Create flag top_neighbourhoods
df_places.loc[df_places.neighbourhood.isin(list(result_1.index)), 'top_neighbourhood'] = 1
df_places.top_neighbourhood.fillna(0, inplace=True)

In [ ]:
plot = df_places.plot(column='top_neighbourhood', cmap='coolwarm')
fig = plot.get_figure()
fig.savefig('figures/neighbourhood_map.png', dpi=600, transparent=True)

# 2. Which elements influence the price?</li>

- Drop outliers in price

In [ ]:
listings_filtered = listings_filtered[listings_filtered.price.apply(lambda x: len(x)) != 10]

- DROP 4 NAN listings reviews in specific categories

In [ ]:
listings_filtered  = listings_filtered [~listings_filtered .review_scores_accuracy.isnull()]

In [ ]:
listings_filtered .reset_index(drop=True, inplace=True)

# Data Prep

In [ ]:
num_cols = [
    'host_since', 'host_response_rate', 'host_acceptance_rate', 'accommodates',
    'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'availability_30',
    'availability_60', 'availability_90', 'availability_365', 'number_of_reviews',
    'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'reviews_per_month', 'price'
    ]

In [ ]:
df_num = listings_filtered[num_cols].copy()

In [ ]:
df_num.head()

In [ ]:
# PREP numerical columns
df_num = utils.calc_delta_day(df_num, 'host_since', True)
df_num = utils.calc_delta_day(df_num, 'last_review', True)
df_num = utils.cast_pct_col(df_num, 'host_response_rate')
df_num = utils.cast_pct_col(df_num, 'host_acceptance_rate')
df_num = utils.cast_currency_col(df_num, 'price')

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
fig = sns.heatmap(ax=ax, data=df_num.corr(), annot=True)

# accomodates, bedrooms and bed are highly correlated with price

- Fill NAs

In [ ]:
df_num.isnull().sum()

- NAN bedrooms are studio type?

In [ ]:
aux = listings[listings.bedrooms.isnull()][['accommodates', 'listing_url']]

In [ ]:
aux.accommodates.value_counts()

In [ ]:
aux[aux.accommodates == 16]

Conclusion: Yes, or places to only rent to make a party

In [ ]:
fill_zero = ['bedrooms', 'beds']
fill_mean = ['host_response_rate', 'host_acceptance_rate']

df_num.loc[:, fill_zero] = df_num[fill_zero].fillna(0)
df_num.loc[:, fill_mean] = df_num[fill_mean].fillna(df_num[fill_mean].mean())

In [ ]:
df_num.isnull().sum().sum()
# Nan problems solved

- Train Test Split

In [ ]:
X = df_num.drop('price', axis=1)
y = df_num.price

In [ ]:
mae_train, mae_test, lmodel = utils.train_lmodel(X, y)

### Categorical Features

In [ ]:
cat_cols = [
    'host_response_time', 'host_is_superhost', 'host_identity_verified',
    'neighbourhood_cleansed', 'room_type', 'amenities', 'instant_bookable'
    ]

In [ ]:
df_cat = listings_filtered[cat_cols].copy()

In [ ]:
df_cat.head()

In [ ]:
df_cat = utils.cast_bool(df_cat, 'host_is_superhost')
df_cat = utils.cast_bool(df_cat, 'host_identity_verified')
df_cat = utils.cast_bool(df_cat,'instant_bookable')
df_cat = utils.flag_room_type(df_cat, 'room_type', True)
df_cat = utils.create_dummies(df_cat, 'host_response_time')
df_cat = utils.create_dummies(df_cat, 'neighbourhood_cleansed')

In [ ]:
df_cat, mlb = utils.processes_mlb(df_cat, 'amenities', True)

In [ ]:
# Putting together num and cat columns
df_final = df_num.join(df_cat)

X = df_final.drop('price', axis=1)
y = df_final.price

In [ ]:
mae_train_cat, mae_test_cat, lmodel_cat = utils.train_lmodel(X, y)

Observation: Overfit due to too many features!

### Feature Selection

In [ ]:
# Create list of possible cuts to vary in minimun percentage
# of positive class distribution in each columns
cuts = list(np.array(range(500, 1, -1))/500)

In [ ]:
mae_trains, mae_tests, num_feats, best_model, reduce_x = utils.select_features(X, y, cuts)

In [ ]:
plt.plot(num_feats, mae_trains)
plt.plot(num_feats, mae_tests.values())

In [ ]:
cols = list(reduce_x.columns)
feature_importance = pd.DataFrame(best_model.coef_, index=list(reduce_x.columns))
feature_importance.columns = ['feature_importance']

In [ ]:
# High positive importance features
feature_importance.sort_values(by='feature_importance', ascending=False).head(10)

In [ ]:
# High negative importance features
feature_importance.sort_values(by='feature_importance').head(5)

# 3. What about economy due to COVID-19 quarantine?

In [ ]:
# Filter always same period in each year
reviews_2020 = revs[revs.date >= '2020-01-01'].copy()
reviews_2019 = revs[(revs.date >= '2019-01-01') & (revs.date <= '2019-10-25')]
reviews_2018 = revs[(revs.date >= '2018-01-01') & (revs.date <= '2018-10-25')]
reviews_2017 = revs[(revs.date >= '2017-01-01') & (revs.date <= '2017-10-25')]
reviews_2016 = revs[(revs.date >= '2016-01-01') & (revs.date <= '2016-10-25')]

In [ ]:
reviews_2020 = utils.create_month(reviews_2020, 'date')
reviews_2019 = utils.create_month(reviews_2019, 'date')
reviews_2018 = utils.create_month(reviews_2018, 'date')
reviews_2017 = utils.create_month(reviews_2017, 'date')
reviews_2016 = utils.create_month(reviews_2016, 'date')

In [ ]:
count_reviews_2020 = utils.count_reviews(reviews_2020, 'month', 2020)
count_reviews_2019 = utils.count_reviews(reviews_2019, 'month', 2019)
count_reviews_2018 = utils.count_reviews(reviews_2018, 'month', 2018)
count_reviews_2017 = utils.count_reviews(reviews_2017, 'month', 2017)
count_reviews_2016 = utils.count_reviews(reviews_2016, 'month', 2016)

In [ ]:
count_concat = pd.concat([count_reviews_2020,
                          count_reviews_2019,
                          count_reviews_2018,
                          count_reviews_2017,
                          count_reviews_2016])

In [ ]:
count_concat.groupby('year').counts.sum()

In [ ]:
g = sns.catplot(
    data=count_reviews_2020, kind="bar",
    x="month", y="counts", hue="year",
    ci="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("Month", "Number of reviews")
g.legend.set_title("")

In [ ]:
result_sec_tri = utils.compare_tri(count_reviews_2019, count_reviews_2020, ['04', '05', '06'])
msg = """In 2020 second trimester, the number of reviews reduced {}% over same period in 2019.""".format(result_sec_tri)
print(msg)

In [ ]:
result_sec_tri = utils.compare_tri(count_reviews_2019, count_reviews_2020, ['07', '08', '09'])
msg = """In 2020 third trimester, the number of reviews reduced {}% over same period in 2019.""".format(result_sec_tri)
print(msg)

In [ ]:
g = sns.catplot(
    data=count_concat, kind="bar",
    x="month", y="counts", hue="year",
    ci="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("Month", "Number of reviews")
g.legend.set_title("")
g.savefig("figures/history_num_reviews.png", dpi=600, transparent=True)

Airbnb shows a growth over the years, including beginning of 2020 in janurary and februrary. However, COVID-19's 
phenomenon impacts hardly the services. In Brazil, quarantine started 24/03/2020. During second trimester of 2020, 
reviews numbers in RIO descresed 82% and third trimester 49% both compared with 2019 in same period. The flow is
getting better but it yet is really far from growth before COVID-19.

### Check specific months in years that are outliars for annual growth behavior

- Aug/2016

In August of 2016, Brazil hosted Olympic Games

- sep/2017

In [ ]:
reviews_2017_sep = reviews_2017[reviews_2017.month == '09']
count_days_2017_sep = utils.count_reviews(reviews_2017_sep, 'date', 2017)
count_days_2017_sep['day'] = count_days_2017_sep.date.str.split('-').apply(lambda x: x[2])

In [ ]:
g = sns.catplot(
    data=count_days_2017_sep, kind="bar",
    x="day", y="counts", hue="year",
    ci="sd", palette="dark", alpha=.6, height=6,
)
g.despine(left=True)
g.set_axis_labels("Month", "Number of reviews")
g.legend.set_title("")

During 15, 16, 17, 21, 22, 23 and 24 on september in 2017, 
happened Rock in Rio, one of the biggest music festival in the world. Apparentely, the lineup in that specif year was really good, because reviews after these days increased heavily.